In [1]:
import csv, sys
import os, glob
import string
from string import ascii_lowercase
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
from nltk.collocations import *
from nltk.probability import FreqDist


output_path = '/home/romina/Documents/work/Speeches_mining/outputs'
input_path = '/home/romina/Documents/work/speeches-segmented/greg-tables'

In [4]:
os.chdir(output_path)
output = open('output.csv', 'r')
data = pd.read_csv(output, encoding='utf-8')
text = pd.DataFrame(data[['text']])
data = list()
for index, content in text.iterrows():
    data.append(content['text'])

In [6]:
print(data[0]['date'])

KeyError: 'date'

In [3]:
print(len(data))
text=list()
text.append(data[0])
text.append(data[1])


272217


In [4]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)
    ##setting up and extending stopwords
stop = stopwords.words('english')
stop.extend(
    ['nevertheless', 'would', 'nether''the', 'in', 'may', 'also', 'zero', 'one', 'two', 'three', 'four', 'five',
     'six', 'seven', 'eight', 'nine', 'ten', 'quot', 'across', 'among', 'beside', 'however', 'yet',
     'within', '1', '2', '3', '4', '5', '6', '7', '8', '9'] + list(ascii_lowercase))

tokens = []
for statement in text:
    words = word_tokenize(statement.lower())
    words_clean = []
    for word in words:
        if word not in stop:
            words_clean.append(stemmer.stem(word))
    tokens.append(words_clean)

In [10]:
n=3
all_trigrams=list()
text_trigram=list()
for token_group in tokens:
    trigram_group = list(nltk.ngrams(token_group, n))
    text_trigram.append(trigram_group)
    for trigram in trigram_group:
        all_trigrams.append(trigram)


2

In [22]:
total=len(all_trigrams)
cutoff = 0.00001 * total
fdist = FreqDist(all_trigrams)
# for k,v in fdist.items():
#     print(k,v)
print(cutoff)
selected_trigrams={k: v for k, v in fdist.items() if v > cutoff}
selected_trigrams=list(selected_trigrams.keys())

0.011170000000000001
[('speaker', 'move', 'suspend'), ('move', 'suspend', 'rule'), ('suspend', 'rule', 'pass'), ('rule', 'pass', 'bill'), ('pass', 'bill', 'hr'), ('bill', 'hr', '3980'), ('hr', '3980', 'provid'), ('3980', 'provid', 'identifi'), ('provid', 'identifi', 'elimin'), ('identifi', 'elimin', 'redund'), ('elimin', 'redund', 'report'), ('redund', 'report', 'requir'), ('report', 'requir', 'develop'), ('requir', 'develop', 'meaning'), ('develop', 'meaning', 'perform'), ('meaning', 'perform', 'metric'), ('perform', 'metric', 'homeland'), ('metric', 'homeland', 'secur'), ('homeland', 'secur', 'prepared'), ('secur', 'prepared', 'grant'), ('prepared', 'grant', 'purpos'), ('grant', 'purpos', 'amend'), ('purpos', 'amend', 'clerk'), ('amend', 'clerk', 'read'), ('clerk', 'read', 'titl'), ('read', 'titl', 'bill'), ('titl', 'bill', 'text'), ('bill', 'text', 'bill'), ('text', 'bill', 'follow'), ('bill', 'follow', 'hr'), ('follow', 'hr', '3980'), ('hr', '3980', 'enact'), ('3980', 'enact', 'sen

In [62]:
binary_matrix = np.zeros(shape=(len(text_trigram), len(selected_trigrams)))
frequency_matrix = np.zeros(shape=(len(text_trigram), len(selected_trigrams)))

for index in range(0, len(text_trigram)):
    for token in text_trigram[index]:
        if token in selected_trigrams:
            binary_matrix[index, selected_trigrams.index(token)] = 1
            frequency_matrix[index, selected_trigrams.index(token)]=text_trigram[index].count(token)

np.savetxt("binary_matrix.csv", binary_matrix, delimiter=",")
np.savetxt("frequency_matrix.csv",frequency_matrix , delimiter=",")
os.chdir(output_path)
f = open('trigrams_position.csv', 'w')
w = csv.writer(f)
w.writerow(['position', 'trigrams'])
for index in range(len(selected_trigrams)):
    w.writerow([index,selected_trigrams[index] ])


In [61]:
from numpy import genfromtxt
os.chdir(output_path)
text1=genfromtxt('binary_matrix_1.csv', delimiter=',')
text2=genfromtxt('binary_matrix.csv', delimiter=',')
print(text1.shape)
for i in range(1008):
    if text1.item(0,i)!=text2.item(0,i):
        print(i,"  ",text1.item(0,i),"  ",text2.item(0,i))
np.array_equal(text1,text2)

(2, 1008)


True